In [5]:
import pandas as pd
import random
import os
import cv2
import imghdr
import time
import numpy as np
import datetime
import pandas as pd
from PIL import Image
import datetime
from tqdm import tqdm

In [113]:
    # Configure where to save the resulting dataset
    save_path_dir = '../data/intermediate/'
    os.makedirs(save_path_dir, exist_ok=True)
    save_path = save_path_dir+'inter_medpix.csv'

    # Read the raw df
    path = '../data/raw/medpix/Dataset_MedPix_V1.xlsx'
    df = pd.read_excel(path)

    # Grab useful columns
    # useful_cols=['ID','Plane',
    #              'Core_Modality','Full_Modality',
    #              'Findings','Case_Diagnosis','Location']
    useful_cols=['Case_URL','ID','Plane',
                 'Core_Modality','Full_Modality',
                 'Caption','Case_Diagnosis','Location']
    
    df=df[useful_cols]

    # Rename columns
    renamings={'ID':'Path',
               'Full_Modality':'Modality',
              'Case_Diagnosis':'Impression',
              'Location':'Anatomy'}
    df.rename(columns=renamings,inplace=True)
    # Now the dataframe contains columns 
    #['Path', 'Plane', 'Core_Modality', 'Modality', 'Caption', 'Impression',
    # 'Anatomy']


    # Drop rows with empty values
    df = df.dropna() 

    # CLEANING ON THE "PLANE" COLUMN-------------------------
    # Consolidating synonims into single types.
    df.Plane.replace('Transverse','Axial',inplace=True)
    df.Plane.replace('Lateral','Sagittal',inplace=True)
    df.Plane.replace('Frontal','Coronal',inplace=True)

    # Keep rows that have a plane values with a frequency higher than 100
    valid_index=df.Plane.value_counts().index[df.Plane.value_counts()>100]
    df=df.loc[df.Plane.isin(valid_index)]

    # Drop whenever plane is equal to particular values.
    df = df.loc[~df.Plane.isin(['NOS - Not specified', #
                          'Other View (see caption)'])]

    # CLEANING ON THE "CORE_MODALITY" COLUMN-------------------
    # Consolidating synonims under the same concept.
    df.Core_Modality.replace('US-D','US',inplace=True)
    df.Core_Modality.replace('CTA','AN',inplace=True)
    df.Core_Modality.replace('MRA','AN',inplace=True)
    df.Core_Modality.replace('Histology','HE',inplace=True)
    df.Core_Modality.replace('PET','PET/NM',inplace=True)
    df.Core_Modality.replace('NM','PET/NM',inplace=True)
    df.Core_Modality.replace('PET-CT','PET/NM',inplace=True)
    df.Core_Modality.replace('MRS','MR',inplace=True)
    
    # Renaming interventional instances to INT
    df.Core_Modality.replace('Interventional','INT',inplace=True) 

    # Keep rows that have a Core_Modality values with a frequency higher than 100
    valid_index=df.Core_Modality.value_counts().index[df.Core_Modality.value_counts()>100]
    df=df.loc[df.Core_Modality.isin(valid_index)]

    # Drop whenever plane is equal to particular values.
    df = df.loc[~df.Core_Modality.isin(['NOS'])]

    # CLEANING ON "FINDINGS" COLUMN------------------------------
    # Eliminate rows that have a "findings" wordcount larger than 100 words. 
    df["Number of Words"] = df["Caption"].apply(lambda n: len(n.split()))
    df=df.loc[df['Number of Words']<=100]

    # CLEANING ON THE "ANATOMY" COLUMN--------------------------
    # Consolidation
    df.Anatomy.replace('Brain and Neuro','Brain',inplace=True)
    df.Anatomy.replace('Nerve, central','Brain',inplace=True)

    df.Anatomy.replace('MSK - Musculoskeletal','Musculoskeletal',inplace=True)
    df.Anatomy.replace('Extremity - Please Select MSK','Musculoskeletal',inplace=True)

    df.Anatomy.replace('Chest, Pulmonary (ex. Heart)','Pulmonary',inplace=True)

    df.Anatomy.replace('Breast and Mammography','Breast',inplace=True)

    df.Anatomy.replace('Abdomen - Generalized','Abdomen',inplace=True)
    df.Anatomy.replace('Gastrointestinal','Abdomen',inplace=True)

    df.Anatomy.replace('Head and Neck (ex. orbit)','Head and Neck',inplace=True)
    df.Anatomy.replace('Eye and Orbit (exclude Ophthalmology)','Head and Neck',inplace=True)

    df.Anatomy.replace('Vascular','Cardiovascular',inplace=True)
    df.Anatomy.replace('Cardiovascular (inc. Heart)','Cardiovascular',inplace=True)

    df.Anatomy.replace('Multisystem','Generalized',inplace=True)

    # Keep rows that have Anatomy values with a frequency higher than 200
    valid_index=df.Anatomy.value_counts().index[df.Anatomy.value_counts()>200]
    df=df.loc[df.Anatomy.isin(valid_index)]

    # CLEANING THE IMPRESSIONS COLUMN---------------------
    # Keep impressions with at most 30 words.
    df["Number of Words"] = df["Impression"].apply(lambda n: len(n.split()))
    df=df.loc[df['Number of Words']<=30]

    # Eliminate the Number of words column
    df.drop(columns='Number of Words').count()

    # CONVERT THE PATH COLUMN INTO THE COMPLETE PATHS--------------------
    prefix = '../data/raw/medpix/Images/'
    suffix = '.jpg'
    df.Path = prefix+df.Path.astype(str)+suffix

    # CREATE THE FULL CAPTIONS COLUMN-------------------------------------
    df['Full_Caption']=df.apply(lambda row: ('<start>'+
                                             ' Core Modality: '+ str(row['Core_Modality'])+
                                             ' Modality: ' + str(row['Modality'])+
                                             ' Plane: ' + str (row['Plane']) +
                                             ' Anatomy: ' + str(row['Anatomy'])+
                                             ' Findings: '+ str(row['Caption'])+
                                             #' Impression: '+ str(row['Impression'])+
                                             ' <end>') ,axis=1)

#      # SPLIT DATA IN TRAIN, VALIDATION AND TEST----------------------------------
#     cases=list(df.Case_URL.unique()) # Make a list of medical cases
#     # Shuffle the list
#     random.seed(1)
#     random.shuffle(cases)

#     # Build the test and validation set with 10% of cases
#     cases=list(df.Case_URL.unique()) # Make a list of medical cases
#     # Shuffle the list
#     random.seed(1)
#     random.shuffle(cases)

#     # Build the test and validation set with 10% of cases
#     test_idx=int(len(cases)*0.05)
#     val_idx=test_idx+int(len(cases)*0.05)

#     test_cases = cases[0:test_idx] # list of test cases
#     val_cases = cases[test_idx:val_idx] # list of validation cases
#     train_cases = cases[val_idx:] # list of training cases

#     df.loc[df.Case_URL.isin(test_cases),'Split']='test'
#     df.loc[df.Case_URL.isin(val_cases),'Split']='validation'
#     df.loc[df.Case_URL.isin(train_cases),'Split']='train'



In [114]:
# SPLIT DATA IN STRATIFIED TRAIN, VALIDATION AND TEST --------------------------
# Choose stratified test set
test_cases=[]
for modality in df.Core_Modality.unique():
# For every modality in the dataframe
    sub_df=df.loc[df.Core_Modality==modality] # Get a df with this modality
     # Get a list of unique cases with this modality. 
    available_cases=list(sub_df.Case_URL.unique())
    chosen_cases= random.sample(available_cases,5) # Choose 5 cases at random.
    test_cases+=chosen_cases # Append chosen cases to test_cases. 
    
# Add test label to all rows corresponding to the chosen test cases. 
df.loc[df.Case_URL.isin(test_cases),'Split']='test'

# Choose stratified validation set from remianing data. 
val_cases=[]
for modality in df.Core_Modality.unique():
# For every modality in the dataframe. 
    # Make a subdataframe with rows corresponding to this modality and e
    # excluding rows already labeled as 'test'
    sub_df=df.loc[(df.Core_Modality==modality)
                  & (df['Split']!='test')]
    available_cases=list(sub_df.Case_URL.unique())
    chosen_cases= random.sample(available_cases,5)
    val_cases+=chosen_cases

# Select the validation df and exclude the validation cases from the rest of 
# the data. 
df.loc[df.Case_URL.isin(val_cases),'Split']='validation'
# Label rows that are not 'test' or 'validation' as 'train'
df.loc[~df.Split.isin(['test','validation']),'Split']='train'


In [115]:
df.Split.value_counts()

train         28071
validation      324
test            296
Name: Split, dtype: int64

In [91]:
cases

['https://medpix.nlm.nih.gov/bycase?id=00784aff-ae01-4ede-92c1-09f7df5a5bba',
 'https://medpix.nlm.nih.gov/bycase?id=a90f0bba-7414-41eb-ab2d-cc2e52de46d3',
 'https://medpix.nlm.nih.gov/bycase?id=753514a0-a130-4968-9ae5-0f98939ed681',
 'https://medpix.nlm.nih.gov/bycase?id=b62c4b59-b4fe-4028-8135-c45e8b074ed9',
 'https://medpix.nlm.nih.gov/bycase?id=f4c98eeb-f7da-45d6-9c3b-e51527c0a86e']

In [87]:
test_cases=[]
df.groupby(['Core_Modality','Case_URL']).count().reset_index().groupby('Core_Modality').count()

,Case_URL,Path,Plane,Modality,Caption,Impression,Anatomy,Number of Words,Full_Caption
Core_Modality,,,,,,,,,
AN,295,295,295,295,295,295,295,295,295
BAS,81,81,81,81,81,81,81,81,81
CT,2484,2484,2484,2484,2484,2484,2484,2484,2484
GR,44,44,44,44,44,44,44,44,44
HE,41,41,41,41,41,41,41,41,41
INT,56,56,56,56,56,56,56,56,56
MR,1925,1925,1925,1925,1925,1925,1925,1925,1925
PET/NM,206,206,206,206,206,206,206,206,206
UGI,77,77,77,77,77,77,77,77,77
